O Schedule e o Schedule manager estavam funcionais, entretando o Schedule não poderia se referir ao manager por uma questão de acoplamento, e é necessário que sempre que uma alteração seja feita no Schedule, o update seja chamado no Schadule manager.
Para isso a saída escolhida foi utilizar o padrão Observer, que resolve esse problema.
Aqui estão a classe de interface para o Observer e o Subject:

In [ ]:
"""
Module that contains the observer interface for the schedule.
"""
from abc import ABC, abstractmethod

class Subject(ABC): ...

class Observer(ABC):
    """
    Observer interface, used to implement the observer pattern in the schedule.
    solve the problem of circular dependencies between the schedule and the schedule management.
    provide a update method that is called when the schedule is updated.
    """
    
    @abstractmethod
    def update(self, subject: Subject) -> None:
        """
        Called when the schedule is updated.

        Arguments:
            schedule -- the schedule that was updated.
        """
        pass

class Subject(ABC):
    """
    Subject interface, used to implement the observer pattern in the schedule.
    solve the problem of circular dependencies between the schedule and the schedule management.
    provide a attach method that is called when the schedule is updated.
    """
    
    @abstractmethod
    def attach(self, observer: Observer) -> None:
        """
        Attach an observer to the subject.

        Arguments:
            observer -- the observer to attach.
        """
        pass

    @abstractmethod
    def detach(self, observer: Observer) -> None:
        """
        Detach an observer from the subject.

        Arguments:
            observer -- the observer to detach.
        """
        pass

    @abstractmethod
    def notify(self) -> None:
        """
        Notify all the observers that the subject has changed.
        """
        pass

Agora necessitamos que essa funcionalidade seja corretamente implementada nas classes, para isso vamos preparar um teste inicial:

In [ ]:
def test_subject_start_with_no_observers(self):
    permissions = {
        'user1': "read"
    }
    schedule = Schedule('schedule_id', 'title', 'description', permissions)
    
    initial_observers_len = len(schedule.__observers)
    self.assertEqual(initial_observers_len, 0)

def test_subject_add_observer(self):
    permissions = {
        'user1': "read"
    }
    schedule = Schedule('schedule_id', 'title', 'description', permissions)
    
    schedule.attach(self.observer)
        observers_len = len(schedule.__observers)
        self.assertEqual(observers_len, 1)

Fazendo o Schedule se comportar como Subject, e adicionando os metodos necessários para passar nos testes

```python
from .schedule_observer import Observer, Subject

class Schedule(Subject):
    def __init__(self, schedule_id: str, title: str, description: str, 
            permissions: dict, elements: [str] = None):
        [...]
        self.__observers = []
    
    [...]
    @property
    def observers(self):
        return self.__observers
    
    @elements.setter
    def elements(self, value):
        if isinstance(value, list) and all(isinstance(i, str) for i in value):
            self.__elements = value
            self.notify() # adicionado o notify
        else:
            raise ValueError("Elements must be a list of strings")

    [...]

    
    def attach(self, observer: Observer) -> None:
        '''
            Attach an observer to the subject.

            Arguments:
                observer -- the observer to attach.
        '''
        self.__observers.append(observer)

    def detach(self, observer: Observer) -> None:
        '''
            Detach an observer from the subject.

            Arguments:
                observer -- the observer to detach.
        '''
        self.__observers.remove(observer)

    def notify(self) -> None:
        '''
            Notify all the observers that the subject has changed.
        '''
        for observer in self.__observers:
            observer.update(self)

```

